In [1]:
import findspark
findspark.init()


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

#Configurar Sesion Spark
spark = SparkSession\
    .builder\
    .master('local[2]')\
    .appName('quake_etl')\
    .config('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.12:2.4.1')\
    .getOrCreate()  